<a href="https://colab.research.google.com/github/qorah/vic-edu-housing-insights/blob/main/vce_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title Cell 1: Import Dependencies, Configure Logging & Create Directories
import os
import logging
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def configure_logging():
    """
    Configures the logging settings for the pipeline.

    Logging is set to INFO level with a format that includes timestamp, log level, and message.
    """
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Logging is configured.")

def create_directories():
    """
    Creates necessary directories for raw data, processed data, and visualizations.

    Ensures that the directories exist; if not, they will be created.
    """
    directories = ['data/raw', 'data/processed', 'visualizations']
    for dir_path in directories:
        os.makedirs(dir_path, exist_ok=True)
        logging.info(f"Directory created or already exists: {dir_path}")

# Execute setup functions
configure_logging()
create_directories()

logging.info("Dependencies imported and directories created successfully.")
print("Dependencies imported and directories created.")





Dependencies imported and directories created.


In [6]:
#@title Cell 2: Data Exploration and Read Data
def explore_excel_structure(file_path):
    """
    Explores the structure of an Excel file by trying different header rows.

    Iterates through potential header rows and logs the sheet names, data shape, and first few columns.
    Returns the header row that appears to structure the data correctly along with a sample of the data.

    Parameters:
        file_path (str): The complete path to the Excel file.

    Returns:
        tuple: (header_row (int), sample_data (pd.DataFrame)) if a valid header is found;
               (None, None) otherwise.
    """
    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
        logging.info(f"Available sheet names: {sheet_names}")
    except Exception as e:
        logging.error(f"Failed to open Excel file at {file_path}: {e}")
        raise e

    # Attempt to identify the correct header row based on data shape and column count
    for header_row in range(0, 15):
        try:
            df = pd.read_excel(file_path, sheet_name=sheet_names[0], header=header_row)
            logging.info(f"Attempt with header_row={header_row}, DataFrame shape: {df.shape}")
            logging.info(f"First few columns: {list(df.columns)[:5]}")
            # Example condition: valid header if more than 10 columns are detected
            if df.shape[1] > 10:
                return header_row, df.head(2)
        except Exception as e:
            logging.error(f"Error reading with header_row={header_row}: {e}")
    logging.error("Unable to determine a valid header row for the Excel file.")
    return None, None

def read_data(file_name, sheet_name=None, header_row=None):
    """
    Reads data from an Excel file stored in the 'data/raw' directory.

    If header_row is not provided, the function will call explore_excel_structure to determine it.
    If sheet_name is not provided, the first sheet is used by default.

    Parameters:
        file_name (str): Name of the Excel file (must be located in 'data/raw').
        sheet_name (str, optional): The sheet name to read. Defaults to the first sheet.
        header_row (int, optional): The row number to use as header. If None, auto-detection is attempted.

    Returns:
        pd.DataFrame: The loaded DataFrame.

    Raises:
        ValueError: If a valid header row cannot be determined.
    """
    file_path = os.path.join('data/raw', file_name)
    logging.info(f"Attempting to read file: {file_path}")

    # Determine the header row if not provided
    if header_row is None:
        header_row, sample = explore_excel_structure(file_path)
        if header_row is None:
            error_msg = "Could not determine an appropriate header row from the Excel file."
            logging.error(error_msg)
            raise ValueError(error_msg)
        else:
            logging.info(f"Detected header row {header_row}. Sample data:\n{sample}")

    # Use the first sheet if sheet_name is not provided
    if sheet_name is None:
        try:
            sheet_name = pd.ExcelFile(file_path).sheet_names[0]
            logging.info(f"No sheet name provided; defaulting to first sheet: {sheet_name}")
        except Exception as e:
            logging.error(f"Error retrieving sheet names: {e}")
            raise e

    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
        logging.info(f"Successfully loaded {file_name} with header_row={header_row}. DataFrame shape: {df.shape}")
        logging.info(f"Column names: {list(df.columns)}")
        return df
    except Exception as e:
        logging.error(f"Error reading data from {file_name}: {e}")
        raise e

# For testing in Colab, adjust the file name as needed.
file_name = "2024SeniorSecondaryCompletionAndAchievementInformation.xlsx"
header, sample = explore_excel_structure(os.path.join('data/raw', file_name))
print("Detected header row:", header)
print("Sample data:\n", sample)




Detected header row: 0
Sample data:
   Senior Secondary Completion and Achievement Information, 2024 Unnamed: 1  \
0                                                NaN                   NaN   
1                                              Notes                   NaN   

  Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   

  Unnamed: 8 Unnamed: 9  ... Unnamed: 13 Unnamed: 14 Unnamed: 15 Unnamed: 16  \
0        NaN        NaN  ...         NaN         NaN         NaN         NaN   
1        NaN        NaN  ...         NaN         NaN         NaN         NaN   

  Unnamed: 17 Unnamed: 18 Unnamed: 19 Unnamed: 20 Unnamed: 21 Unnamed: 22  
0         NaN         NaN         NaN         NaN         NaN         NaN  
1         NaN         NaN         NaN         NaN         NaN         NaN  

[2 rows x 23 columns]


In [ ]:
#@title Cell 3: Data Cleaning
def clean_data(df):
    """
    Clean the DataFrame by handling special values and converting data types.

    Steps:
      - Display unique values in columns (if few) for understanding.
      - Use the first column as the school name column.
      - Identify and process a "Small School" column if available.
      - Replace special characters/codes (e.g., '-', '<4', 'I/D') with 0.
      - Convert columns to numeric types when applicable.
      - Identify key columns such as VCE median score and high scores percentage.
      - Mark selective schools by updating the 'Sector' column.

    Returns:
      cleaned_df: The cleaned DataFrame.
      school_col: The column used as the school name.
      vce_median_col: The column representing the VCE median score.
      high_scores_col: The column representing high scores percentage (if found).
    """
    cleaned_df = df.copy()

    # Display unique values in each column if there are few unique values
    for col in cleaned_df.columns:
        unique_vals = cleaned_df[col].unique()
        if len(unique_vals) < 10:
            logging.info(f"Unique values in {col}: {unique_vals}")

    # Use the first column as the school name column
    school_col = cleaned_df.columns[0]
    logging.info(f"Using {school_col} as school name column")

    # Check for a "Small School" column (might be named differently)
    small_school_col = None
    for col in cleaned_df.columns:
        if isinstance(col, str) and 'small' in col.lower():
            small_school_col = col
            break
        elif col == 'Unnamed: 1':  # Possibility based on file structure
            if '*' in cleaned_df[col].values:
                small_school_col = col
                break

    if small_school_col:
        logging.info(f"Found Small School column: {small_school_col}")
        # Remove header rows that contain column descriptions
        cleaned_df = cleaned_df[~cleaned_df[small_school_col].astype(str).str.contains('Small School', na=False)]
        # Replace '*' with 1 and convert to integer
        cleaned_df[small_school_col] = cleaned_df[small_school_col].replace('*', 1)
        cleaned_df[small_school_col] = cleaned_df[small_school_col].fillna(0).astype(int)
        # Drop small schools if indicated
        cleaned_df = cleaned_df[cleaned_df[small_school_col] == 0]
    else:
        logging.warning("Small School column not found")

    # Replace '-' with 0
    cleaned_df = cleaned_df.replace('-', 0)

    # Replace patterns like '-.1', '-.2', etc. with 0
    for col in cleaned_df.columns:
        cleaned_df[col] = cleaned_df[col].astype(str).replace(r'-\.\d+', '0', regex=True)

    # Replace 'I/D' (indicating no enrollment) with 0
    cleaned_df = cleaned_df.replace('I/D', 0)

    # Replace '<4' and '< 4' with 0
    cleaned_df = cleaned_df.replace('<4', 0)
    cleaned_df = cleaned_df.replace('< 4', 0)

    # Check for an IB column and process it
    ib_col = None
    for col in cleaned_df.columns:
        if isinstance(col, str) and ('ib' in col.lower() or 'international' in col.lower()):
            ib_col = col
            break
        elif col == 'Unnamed: 8':
            if 'Y' in cleaned_df[col].values:
                ib_col = col
                break
    if ib_col:
        logging.info(f"Found IB column: {ib_col}")
        cleaned_df[ib_col] = cleaned_df[ib_col].replace('Y', 1).fillna(0)

    # Check for an Adult column and filter out adult sectors
    adult_col = None
    for col in cleaned_df.columns:
        if isinstance(col, str) and 'adult' in col.lower():
            adult_col = col
            break
    if adult_col:
        logging.info(f"Found Adult column: {adult_col}")
        cleaned_df[adult_col] = cleaned_df[adult_col].replace('A', 1).fillna(0)
        cleaned_df = cleaned_df[cleaned_df[adult_col] == 0]

    # Add a 'Sector' column; default value is 'REGULAR'
    cleaned_df['Sector'] = 'REGULAR'

    # Convert columns to numeric where applicable
    numeric_cols = []
    for col in cleaned_df.columns:
        if col in [school_col, 'Sector'] or (isinstance(col, str) and 'local' in col.lower()):
            continue
        try:
            pd.to_numeric(cleaned_df[col], errors='raise')
            numeric_cols.append(col)
        except:
            numeric_count = pd.to_numeric(cleaned_df[col], errors='coerce').notna().sum()
            if numeric_count > len(cleaned_df) * 0.5:
                numeric_cols.append(col)

    logging.info(f"Identified numeric columns: {numeric_cols}")
    for col in numeric_cols:
        cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce').fillna(0)

    # Identify the VCE median score column
    vce_median_col = None
    for col in cleaned_df.columns:
        if isinstance(col, str) and 'median' in col.lower() and 'vce' in col.lower():
            vce_median_col = col
            break
        elif isinstance(col, str) and 'median' in col.lower() and 'score' in col.lower():
            vce_median_col = col
            break
    if not vce_median_col:
        # Fallback: look for a numeric column with typical VCE score values (20-40)
        for col in numeric_cols:
            col_mean = cleaned_df[col].mean()
            if 20 <= col_mean <= 40:
                vce_median_col = col
                logging.info(f"Identified potential VCE median score column: {col} (mean: {col_mean})")
                break
    if vce_median_col:
        logging.info(f"Using {vce_median_col} as VCE median score column")
        # Filter out records with a 0 median score
        cleaned_df = cleaned_df[cleaned_df[vce_median_col] > 0]

    # Identify selective schools and mark their 'Sector' as 'SELECTIVE'
    selective_schools = [
        'MELBOURNE HIGH SCHOOL',
        'MACROBERTSON GIRLS HIGH SCHOOL',
        'NOSSAL HIGH SCHOOL',
        'SUZANNE CORY HIGH SCHOOL',
        'JOHN MONASH SCIENCE SCHOOL'
    ]
    for school in selective_schools:
        for match_type in ['exact', 'contains', 'partial']:
            if match_type == 'exact':
                matches = cleaned_df[school_col].astype(str).str.upper() == school
            elif match_type == 'contains':
                matches = cleaned_df[school_col].astype(str).str.upper().str.contains(school, na=False)
            else:
                key_parts = school.split()
                if len(key_parts) >= 2:
                    pattern = '|'.join(key_parts)
                    matches = cleaned_df[school_col].astype(str).str.upper().str.contains(pattern, na=False)
                else:
                    continue
            if matches.any():
                logging.info(f"Found selective school ({match_type} match): {school}")
                cleaned_df.loc[matches, 'Sector'] = 'SELECTIVE'
                break

    # Additional pattern matching for selective schools
    specific_patterns = {
        'Melbourne High': 'MELBOURNE HIGH',
        'MacRobertson': 'MACROBERTSON',
        'Mac.?Robertson': 'MACROBERTSON',
        'Nossal High': 'NOSSAL',
        'Suzanne Cory': 'SUZANNE CORY',
        'John Monash': 'JOHN MONASH'
    }
    for name, pattern in specific_patterns.items():
        matches = cleaned_df[school_col].astype(str).str.contains(pattern, case=False, na=False, regex=True)
        if matches.any():
            logging.info(f"Found selective school using pattern: {pattern}")
            cleaned_df.loc[matches, 'Sector'] = 'SELECTIVE'

    # Fallback: Manual keyword check for selective schools
    school_keywords = {
        'MELBOURNE HIGH': ['MELBOURNE', 'HIGH'],
        'MACROBERTSON': ['MACROBERTSON', 'MAC ROBERTSON', 'GIRLS'],
        'NOSSAL': ['NOSSAL'],
        'SUZANNE CORY': ['SUZANNE', 'CORY'],
        'JOHN MONASH': ['JOHN', 'MONASH', 'SCIENCE']
    }
    for idx, row in cleaned_df.iterrows():
        school_name = str(row[school_col]).upper()
        for school_key, keywords in school_keywords.items():
            if any(keyword in school_name for keyword in keywords):
                logging.info(f"Found potential selective school by keyword: {school_name}")
                cleaned_df.loc[idx, 'Sector'] = 'SELECTIVE'

    # Identify high scores percentage column
    high_scores_col = None
    for col in cleaned_df.columns:
        if isinstance(col, str) and '40' in col and ('over' in col.lower() or 'plus' in col.lower()):
            high_scores_col = col
            break
    if not high_scores_col:
        for col in numeric_cols:
            if col != vce_median_col and col != 'Sector' and col != school_col:
                col_values = cleaned_df[col].dropna()
                if len(col_values) > 0 and col_values.min() >= 0 and col_values.max() <= 100:
                    if col_values.mean() < 30:
                        high_scores_col = col
                        logging.info(f"Identified potential high scores percentage column: {col} (mean: {col_values.mean()})")
                        break
    if high_scores_col:
        logging.info(f"Using {high_scores_col} as high scores percentage column")

    return cleaned_df, school_col, vce_median_col, high_scores_col

# To test this cell, after loading a DataFrame with read_data(), run:
df = read_data("/content/data/raw/2024SeniorSecondaryCompletionAndAchievementInformation.xlsx")
cleaned_df, school_col, vce_median_col, high_scores_col = clean_data(df)
print("Cleaned DataFrame head:\n", cleaned_df.head())


Cleaned DataFrame head:
    Senior Secondary Completion and Achievement Information, 2024  Unnamed: 1  \
10                         Academy of Mary Immaculate                      0   
14                                     Aitken College                      0   
15                                    Al Iman College                      0   
16                                  Al Siraat College                      0   
17                                   Al-Taqwa College                      0   

      Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5 Unnamed: 6 Unnamed: 7  \
10       FITZROY        34.0         8.0         2.0        nan        nan   
14     GREENVALE        41.0        14.0         0.0          Y        nan   
15  MELTON SOUTH        17.0         0.0         0.0        nan        nan   
16        EPPING        20.0         7.0         2.0        nan        nan   
17     TRUGANINA        32.0        12.0         0.0          Y        nan   

   Unnamed: 8 Unnamed: 9 

<ipython-input-53-b4ca2eeaec56>:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_df[small_school_col] = cleaned_df[small_school_col].replace('*', 1)
<ipython-input-53-b4ca2eeaec56>:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_df = cleaned_df.replace('-', 0)


In [ ]:
#@title Cell 4: Data Analysis
def analyze_data_custom(df, school_col, vce_median_col, high_scores_col=None):
    """
    Perform data analysis:
      - Log total number of schools.
      - Log counts by Sector.
      - Compute and log statistics for VCE median scores by Sector.
      - If available, compute and log statistics for high scores percentage by Sector.
      - List and log the top 20 schools by VCE median score.

    Returns:
      sector_stats: DataFrame with VCE median score stats by Sector.
      sector_high_stats: DataFrame with high scores percentage stats by Sector (if available).
      top_schools: DataFrame of the top 20 schools.
    """
    logging.info(f"Total schools after cleaning: {len(df)}")

    sector_counts = df['Sector'].value_counts()
    logging.info(f"Schools by sector:\n{sector_counts}")

    sector_stats = df.groupby('Sector')[vce_median_col].agg(['count', 'mean', 'median', 'std'])
    logging.info(f"VCE Median Score by Sector:\n{sector_stats}")

    if high_scores_col:
        sector_high_stats = df.groupby('Sector')[high_scores_col].agg(['count', 'mean', 'median', 'std'])
        logging.info(f"High Scores Percentage by Sector:\n{sector_high_stats}")
    else:
        sector_high_stats = None

    top_schools = df.sort_values(vce_median_col, ascending=False).head(20)
    if high_scores_col:
        logging.info(f"Top 20 schools by VCE Median Score:\n{top_schools[[school_col, vce_median_col, high_scores_col, 'Sector']]}")
    else:
        logging.info(f"Top 20 schools by VCE Median Score:\n{top_schools[[school_col, vce_median_col, 'Sector']]}")

    return sector_stats, sector_high_stats, top_schools

# To test this cell, run analysis on the cleaned DataFrame:
stats, high_stats, top20 = analyze_data_custom(cleaned_df, school_col, vce_median_col, high_scores_col)
print("Sector Stats:\n", stats)


Sector Stats:
            count       mean  median       std
Sector                                       
REGULAR      406  28.261084    28.0  3.260813
SELECTIVE    109  30.807339    31.0  3.568061


In [ ]:
#@title Cell 5: Data Visualization
def create_visualizations_custom(df, school_col, vce_median_col, high_scores_col=None):
    """
    Create visualizations:
      1. Boxplot: Distribution of VCE median scores by Sector.
      2. If available, boxplot: Distribution of high scores percentage by Sector.
      3. If available, scatter plot: Correlation between VCE median and high scores percentage.
      4. Bar plot: Top 20 schools by VCE median score.
      5. Histogram: Distribution of VCE median scores.
    """
    sns.set(style="whitegrid")

    # 1. Boxplot for VCE median scores by Sector
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Sector', y=vce_median_col, data=df)
    plt.title('Distribution of VCE Median Scores by Sector')
    plt.savefig('visualizations/vce_scores_by_sector.png')
    plt.close()

    if high_scores_col:
        # 2. Boxplot for high scores percentage by Sector
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='Sector', y=high_scores_col, data=df)
        plt.title('Distribution of VCE High Scores (40+) by Sector')
        plt.savefig('visualizations/high_scores_by_sector.png')
        plt.close()

        # 3. Scatter plot for correlation between VCE median and high scores percentage
        plt.figure(figsize=(8, 8))
        sns.scatterplot(x=vce_median_col, y=high_scores_col, hue='Sector', data=df)
        plt.title('Correlation: VCE Median Score vs. High Scores Percentage')
        plt.savefig('visualizations/score_correlation.png')
        plt.close()

    # 4. Bar plot of top 20 schools by VCE median score
    top_schools = df.sort_values(vce_median_col, ascending=False).head(20)
    plt.figure(figsize=(12, 8))
    colors = ['red' if sector == 'SELECTIVE' else 'blue' for sector in top_schools['Sector']]
    plt.barh(top_schools[school_col], top_schools[vce_median_col], color=colors)
    plt.xlabel('VCE Median Score')
    plt.ylabel('School')
    plt.title('Top 20 Schools by VCE Median Score')
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='red', label='SELECTIVE'), Patch(facecolor='blue', label='REGULAR')]
    plt.legend(handles=legend_elements)
    plt.tight_layout()
    plt.savefig('visualizations/top_schools.png')
    plt.close()

    # 5. Histogram of VCE median scores
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=vce_median_col, bins=20, kde=True)
    plt.title('Histogram of VCE Median Scores')
    plt.xlabel('VCE Median Score')
    plt.ylabel('Frequency')
    plt.savefig('visualizations/vce_median_histogram.png')
    plt.close()

    logging.info("Visualizations created and saved in the 'visualizations' directory.")


create_visualizations_custom(cleaned_df, school_col, vce_median_col, high_scores_col)


In [ ]:
#@title Cell 6: Run the Pipeline in Colab and Fix Final Headers
# In Google Colab, use the file uploader to upload your Excel file.
try:
    from google.colab import files
    uploaded = files.upload()  # Prompts for file upload
    uploaded_file_name = list(uploaded.keys())[0]
    # Save the uploaded file to the data/raw directory
    with open(os.path.join('data/raw', uploaded_file_name), 'wb') as f:
        f.write(uploaded[uploaded_file_name])
    file_name = uploaded_file_name
    logging.info(f"Uploaded file: {file_name}")
except ImportError:
    # If not in Colab, use a local file name
    file_name = 'example.xlsx'
    logging.info(f"Using local file: {file_name}")

def main_pipeline(file_name):
    """
    Main pipeline to run the complete process:
      - Read the Excel data.
      - Clean the data.
      - Analyze the data.
      - Create visualizations.
      - Fix headers and save the cleaned data.
    """
    df = read_data(file_name)
    df_cleaned, school_col, vce_median_col, high_scores_col = clean_data(df)

    # Optionally run analysis and visualization:
    analyze_data_custom(df_cleaned, school_col, vce_median_col, high_scores_col)
    create_visualizations_custom(df_cleaned, school_col, vce_median_col, high_scores_col)

    # Fix final headers
    desired_headers = [
        "School",
        "Small School",
        "Locality",
        "VCE/VM/VET Programs",
        "VET Certificates",
        "HES Enrolments",
        "VM Enrolments",
        "VPC Enrolments",
        "IB Diploma Enrolments",
        "NHT Enrolments",
        "Students in VCE/VM/VET",
        "Students in VET",
        "Students in SBAT",
        "Students in HES",
        "VTAC Applications",
        "VCE Completions",
        "VM Awards",
        "VCE (Bacc) Awards",
        "VET Competency Completion",
        "HES Completion",
        "Median VCE Score",
        "Study Scores 40+",
        "VPC Awards",
        "REGULAR"
    ]
    if df_cleaned.shape[1] >= len(desired_headers):
        # Take the first N columns and assign the desired headers
        df_cleaned = df_cleaned.iloc[:, :len(desired_headers)]
        df_cleaned.columns = desired_headers
    else:
        logging.error("Cleaned data does not have enough columns to match desired headers.")

    # Save the cleaned data with the fixed headers
    df_cleaned.to_csv('data/processed/cleaned_data.csv', index=False)
    logging.info("Processing complete. Cleaned data saved to 'data/processed/cleaned_data.csv'.")

# To run the full pipeline, simply execute:
if __name__ == "__main__":
    main_pipeline(file_name)



Saving 2024SeniorSecondaryCompletionAndAchievementInformation.xlsx to 2024SeniorSecondaryCompletionAndAchievementInformation.xlsx


<ipython-input-53-b4ca2eeaec56>:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_df[small_school_col] = cleaned_df[small_school_col].replace('*', 1)
<ipython-input-53-b4ca2eeaec56>:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_df = cleaned_df.replace('-', 0)
